<a href="https://colab.research.google.com/github/abhianand7/LLMChat/blob/master/langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain
!pip install sentence-transformers
!pip install pypdf
!pip install chromadb
!pip install -q -U bitsandbytes
!pip install ctransformers[cuda]
!pip install safetensors
!pip install accelerate

In [7]:
from langchain.document_loaders import PyPDFLoader

In [8]:
pdf_loader = PyPDFLoader('/content/drive/MyDrive/ResearchProjects/LLMChat/Rsys_India_EmployeeHandbook_01_Apr_2019_Ver3.1 1.pdf')

In [9]:
pages = pdf_loader.load()

In [10]:
print(len(pages))

33


In [11]:
print(pages[0].metadata)

{'source': '/content/drive/MyDrive/ResearchProjects/LLMChat/Rsys_India_EmployeeHandbook_01_Apr_2019_Ver3.1 1.pdf', 'page': 0}


In [12]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=150)

In [13]:
splits = text_splitter.split_documents(pages)

In [ ]:
from langchain.vectorstores import chroma
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

model_name = "mistralai/Mistral-7B-v0.1"
model_kwargs = {'device': 'gpu'}
encode_kwargs = {'normalize_embeddings': False}
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

In [3]:
!pip freeze | grep langchain

langchain==0.0.349
langchain-community==0.0.1
langchain-core==0.0.13


Another Try

In [1]:
import torch
from transformers import BitsAndBytesConfig
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

In [2]:
model_id = "mistralai/Mistral-7B-Instruct-v0.1"

In [3]:
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
model4bit = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", quantization_config=quantization_config)
tokenizer = AutoTokenizer.from_pretrained(model_id)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
pipeline = pipeline(
    "text-generation",
    model=model4bit,
    tokenizer=tokenizer,
    use_cache=True,
    device_map="auto",
    max_length=1500,
    do_sample=True,
    top_k=5,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id= tokenizer.eos_token_id
)

In [5]:
from langchain import HuggingFacePipeline

llm = HuggingFacePipeline(pipeline=pipeline)

In [6]:
from langchain import PromptTemplate, LLMChain

template = """[INST] You are a helpful, respectful and honest assistant. Answer exactly in few words from the context
Answer the question below from the context below:
{context}
{question} [/INST]
"""
question_p = """What is the date for the announcement"""
context_p = """ On August 10, it was announced that its subsidiary, JSW Neo Energy, has agreed to acquire a portfolio encompassing 1753 megawatts of renewable energy generation capacity from Mytrah Energy India Pvt Ltd for Rs 10,530 crore."""

prompt = PromptTemplate(template=template, input_variables=["question", "context"])
llm_chain = LLMChain(prompt=prompt, llm=llm)
response = llm_chain.run({"question": question_p, "context": context_p})

In [7]:
response

'\nThe date for the announcement was August 10.'

In [8]:
import chromadb
from chromadb.config import Settings
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma

In [9]:
from langchain.document_loaders import PyPDFLoader
pdf_loader = PyPDFLoader('/content/drive/MyDrive/ResearchProjects/LLMChat/Rsys_India_EmployeeHandbook_01_Apr_2019_Ver3.1 1.pdf')
pages = pdf_loader.load()

In [10]:
print(len(pages))

33


In [11]:
print(pages[0].metadata)

{'source': '/content/drive/MyDrive/ResearchProjects/LLMChat/Rsys_India_EmployeeHandbook_01_Apr_2019_Ver3.1 1.pdf', 'page': 0}


In [12]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=150)
splits = text_splitter.split_documents(pages)

In [13]:
st_model_id = "sentence-transformers/all-mpnet-base-v2"

In [14]:
model_kwargs = {"device": "cuda"}

In [15]:
embedding_model = HuggingFaceEmbeddings(model_name=st_model_id, model_kwargs=model_kwargs)

In [16]:
chroma_db = Chroma.from_documents(documents=splits, embedding=embedding_model)

In [17]:
retriever = chroma_db.as_retriever()

In [18]:
retriever_qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever, verbose=True)

In [40]:
text_query = "how does the performance of an employee analysed in Radisys"
retriever_qa.run(text_query)



> Entering new RetrievalQA chain...


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(



> Finished chain.


" The performance analysis of an employee at Radisys is done on a regular basis, with the employee's manager tracking the progress vis-à-vis the set goals on a monthly basis and providing timely feedback. The formal review process includes a mid-term review called 'Positive Touch Base', followed by an Annual Performance review. The Annual salary revision is based on the Annual Performance review.\n\nAdditionally, the employee's developmental needs are identified and a robust training system is developed to build the employee's skills and capabilities. The company recognizes excellent performance and achievement through rewards that are creative, flexible, and meaningful. Finally, the company is an equal opportunity employer and ensures that all employment decisions and personnel actions are administered in a nondiscriminatory manner, consistent with the principles of equal opportunity."